* 여행스타일1: 자연 vs 도시(1-7범위)
* 여행스타일2: 숙박 vs 당일(1-7범위)
* 여행스타일3: 새로운 vs 익숙한(1-7범위)
* 여행스타일4: 휴양/휴식 vs 체험활동

In [ ]:
import joblib
model = joblib.load('/content/gbr_model.pkl')
master_visit_all = pd.read_csv('/content/master_visit_all.csv')

In [ ]:
testing = pd.DataFrame({
    'GENDER': [1],
    'AGE_GRP': [20],
    'TRAVEL_STYL_1': [2],
    'TRAVEL_STYL_2': [2],
    'TRAVEL_STYL_3': [2],
    'TRAVEL_STYL_4': [2]
})

In [ ]:
# 3번 문항 range 수정
testing['TRAVEL_STYL_3'] = testing['TRAVEL_STYL_3'].replace({1:'1', 2:'1', 3:'2', 4:'3', 5:'3'})
testing['TRAVEL_STYL_3'] = testing['TRAVEL_STYL_3'].astype(int)

In [ ]:
# 유저 특성에 대해서 여행지를 덧붙여서 oversampling
visit_areas = master_visit_all['VISIT_AREA_NM'].drop_duplicates().dropna(axis=0).tolist()

repeated_visits = np.tile(visit_areas, len(testing))

testing = {
    'GENDER': np.repeat(testing['GENDER'], len(visit_areas)),
    'AGE_GRP': np.repeat(testing['AGE_GRP'], len(visit_areas)),
    'TRAVEL_STYL_1': np.repeat(testing['TRAVEL_STYL_1'], len(visit_areas)),
    'TRAVEL_STYL_2': np.repeat(testing['TRAVEL_STYL_2'], len(visit_areas)),
    'TRAVEL_STYL_3': np.repeat(testing['TRAVEL_STYL_3'], len(visit_areas)),
    'TRAVEL_STYL_4': np.repeat(testing['TRAVEL_STYL_4'], len(visit_areas)),
    'VISIT_AREA_NM': repeated_visits
}

testing = pd.DataFrame(testing).reset_index(drop=True).drop_duplicates()

In [ ]:
scaler = MinMaxScaler()

result = predict_model(model, data = testing)
result['output']= scaler.fit_transform(result[['prediction_label']])

result = result[['VISIT_AREA_NM', 'output']]
result.sort_values(by='output', ascending=False).head(20)

In [ ]:
result.to_csv('result.csv')

In [ ]:
import json

result = result.to_dict(orient='records')

with open('result.json', 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=4)